In [144]:
import math
import plotly
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import scipy as sc
import networkx as nx
from itertools import *

In [181]:
name = "tree-8n.xml"
#G = nx.read_graphml("./%s" % name)
Skeleton = nx.Graph()
Tree = nx.Graph()
fig = go.Figure()
Faces = []
R = {}  #иснтуркции по наложения для оптимизации

G = nx.Graph()
G.add_nodes_from([i for i in range(8)])
edges = [
  (0,1), (0,5), (5,6), (5,7),
  (1,2), (2,3), (2,4)
]
G.add_edges_from(edges)

skeleton_nodes = [i for i in range(12)]
skeleton_edges = [
  (8,9),(9,10),(10,11),
  (7,8),(6,8),(0,9),(9,5),(4,10),(10,1),(3,11),(11,2)
]
skeleton_position = [
  (0,0),(20,0),(20,20),(15,20),(15,5),(5,5),(5,20),
  (0,20),(2.5,17,5),(2.5,2.5),(17.5,2.5),(17.5,17.5)
]
skeleton_border = [
  (0,1),(1,2),(2,3),(3,4),(4,5),(5,6),(6,7),(7,0)
]
skeleton_x = []

class my_node():
  def __init__(self, x=0, y=0):
    self.x = x
    self.y = y
    self.childs = []
    self.parents = []
  def __repr__(self):
    s = "(position=({},{}), parents={}, childs={})\n".format(
      self.x, self.y, self.parents, self.childs)
    return s 

class sk_node(my_node):
  def __init__(self, x=0, y=0):
    self.weight = 0
    super().__init__(x,y)

class Face():
  def __init__(self, nodes, s):
    self.nodes = nodes
    self.s = s
  def __repr__(self):
    s = "(S={}, nodes={})".format(self.s, self.nodes)
    return s

def add_edge(Graph, i, j):
  Graph[i].childs.append(j)
  Graph[j].parents.append(i)
  Graph[i].parents.append(j)
  Graph[j].childs.append(i)
  return

def add_edges_from(Graph, edges):
  for edge in edges:
    add_edge(Graph, edge[0], edge[1])
  return

def create_tree(OutGraph, Graph):
  nodes_dict = {}
  number = 0
  pos = {}
  for node in OutGraph.nodes():
    pos[number] = [0,0]
    nodes_dict[node] = number
    Graph.add_node(number)
    number += 1
  for edge in OutGraph.edges():
    v0 = nodes_dict[edge[0]]
    v1 = nodes_dict[edge[1]]
    Graph.add_edge(v0, v1)
  nx.set_edge_attributes(Graph, 0, 'weight')
  nx.set_node_attributes(Graph, pos, 'pos')
  return

def create_skeleton(Graph, nodes, edges, positions):
  Graph.add_nodes_from(nodes)
  tuple_weight_edges = [(edge[0], edge[1], 0) for edge in edges]
  Graph.add_weighted_edges_from(tuple_weight_edges, weight='weight')
  pos = {}
  for node in nodes:
    pos[node] = list(positions[node])
  nx.set_node_attributes(Graph, pos, 'pos')
  nx.set_node_attributes(Graph, 0, 'weight')
  for p in positions:
    skeleton_x.append(p[0])
    skeleton_x.append(p[1])
  return

def draw_graph(figure, Graph, c):
  for edge in Graph.edges():  #добавление ребер на холст
    v1 = get_pos(Graph, edge[0])
    v2 = get_pos(Graph, edge[1])
    figure.add_trace(go.Scatter(
        x = [v1[0], v2[0]],
        y = [v1[1], v2[1]],
        line = dict(color=c, width=1)
    ))

  for node in Graph.nodes():  #добавление вершин на холст
    figure.add_trace(go.Scatter(
      x = [Graph.nodes[node]['pos'][0]], 
      y = [Graph.nodes[node]['pos'][1]], 
      mode = 'markers',
      marker=dict(size=[10], color=c)
      ))
  return

def draw_border(figure):
  x = [-1,21,21,14,14,6,6,-1,-1]
  y = [-1,-1,21,21,6,6,21,21,-1]
  figure.add_trace(go.Scatter(
      x=x,
      y=y,
      line=dict(color='black', width=3)
  ))
  return

def get_pos(Graph, v):
  return Graph.nodes[v]['pos']

def len_e(a):
  l= len(a)
  u = sum([x*x for x in a])
  return math.sqrt(u)

def angle(b, center):
  a = b.copy()
  a[0] -= center[0]
  a[1] -= center[1]
  if len_e(a) == 0:
    return 0
  an = math.asin(a[1] / len_e(a))
  if a[0] > 0:
    if an < 0:
      return  2 * math.pi + an
    else:
      return an
  else:
    return math.pi - an 
  
def get_Face_from_edge(Graph, edge):
  path = nx.shortest_path(Graph, source = edge[0], target = edge[1])
  center = [0.0, 0.0]
  for node in path:
    center[0] += get_pos(Graph, node)[0] / len(path)
    center[1] += get_pos(Graph, node)[1] / len(path)
  path.sort(key = lambda x: angle(get_pos(Graph, x), center))
  square = 0.0
  path.append(path[0])
  for i in range(len(path) - 1):
    square += get_pos(Graph, path[i])[0] * get_pos(Graph, path[i + 1])[1] - get_pos(Graph, path[i + 1])[0] * get_pos(Graph, path[i])[1]
  return path[:-1], square / 2

def create_Faces(Graph, edges):
  for edge in edges:
    path, square = get_Face_from_edge(Graph, edge)
    Faces.append(Face(path, square))
  Faces[-1].s = Faces[1].s
  Faces[-2].s = Faces[2].s
  Faces[-3].s = Faces[3].s
  return

def W_IA(v):
  return sum([face.s for face in Faces if v in face.nodes])

def CloserSet(i,j, Graph):
  closer_set = []
  for v in Graph.nodes():
    dist_v_i = nx.shortest_path_length(Graph, source=v, target=i)
    dist_v_j = nx.shortest_path_length(Graph, source=v, target=j)
    if dist_v_i < dist_v_j:
      closer_set.append(v)
  return closer_set

def W_CS(Graph, i=None, j=None):
  if i is None and j is None:
    return sum([W_IA(m) for m in Graph.nodes()])
  else:
    return sum([W_IA(m) for m in CloserSet(i, j, Graph)])

def W_CS_tree(Graph, i=None, j=None):
  if i is None and j is None:
    return len(Graph.nodes())
  else:
    return len(CloserSet(i, j, Graph))

def W_E(edge, Graph):
  W_cs_i_j = W_CS(Graph, edge[0], edge[1])
  W_cs_j_i = W_CS(Graph, edge[1], edge[0])
  return abs(W_cs_i_j - W_cs_j_i)

def W_E_tree(edge, Graph):
  n1 = W_CS_tree(Graph, edge[0], edge[1])
  n2 = W_CS_tree(Graph, edge[1], edge[0])
  return abs(n1 - n2)

#def PathSet(i,j, Graph):
#  path = nx.shortest_path(Tree, source=0, target=3)[1:-1].copy()
#  return path

def clique(Graph, i):
  return [node for node in Graph.nodes() if nx.has_path(Graph, node, i)]

def crossing_edges(edge1, edge2):
  for i in edge1:
    for j in edge2:
      if i == j:
        return i
  return -1

def mapping(tree, skeleton, rule):
  #print("tree:", tree.nodes())
  #print("skeleton:", skeleton.nodes())
  #print()
  for edge in skeleton.edges():
    skeleton.edges[edge]['weight'] = W_E(edge, skeleton)

  for edge in tree.edges():
    tree.edges[edge]['weight'] = W_E_tree(edge, tree)

  min_sk_weight = min([skeleton.edges[edge]['weight'] for edge 
    in skeleton.edges()], default=0)
  middle_sk_edge = [edge for edge in
    skeleton.edges() if skeleton.edges[edge]['weight'] == min_sk_weight]
  
  min_tree_weight = min([tree.edges[edge]['weight'] for edge 
    in tree.edges()], default=0)
  middle_tree_edge = [edge for edge in tree.edges() 
    if tree.edges[edge]['weight'] == min_tree_weight]

  #case 0
  if len(middle_sk_edge) == 0 and len(middle_tree_edge) == 0:
    #print("1 node, 1 node")
    rule[list(tree)[0]] = list(skeleton)[0]
    return

  if len(middle_sk_edge) == 0 and len(middle_tree_edge) > 0:
    #print("several edges, 1 node")    
    rule[middle_tree_edge[0][0]] = list(skeleton)[0]
    return

  if len(middle_sk_edge) > 0 and len(middle_tree_edge) == 0:
    #print("1 node, several edges")
    number = 0
    edge = middle_sk_edge[0]
    if W_IA(edge[0]) < W_IA(edge[1]):
      number = 1
    rule[list(tree)[0]] = middle_sk_edge[0][number]
    return

  #case 1
  if len(middle_sk_edge) == 1 and len(middle_tree_edge) == 1:
    #print("middle edge, middle edge")
    u, v = middle_tree_edge[0][0], middle_tree_edge[0][1]
    k, l = middle_sk_edge[0][0], middle_sk_edge[0][1]
    T_u = tree.subgraph(CloserSet(u, v, tree))  
    T_v = tree.subgraph(CloserSet(v, u, tree))
    S_k = skeleton.subgraph(CloserSet(k, l, skeleton))
    S_l = skeleton.subgraph(CloserSet(l, k, skeleton))
    d1 = W_CS_tree(tree,u,v)/W_CS(skeleton,k,l) - W_CS_tree(tree, v,u)/W_CS(skeleton,l,k)
    d2 = W_CS_tree(tree,v,u)/W_CS(skeleton,k,l) - W_CS_tree(tree, u,v)/W_CS(skeleton,l,k)

    if abs(d1) > abs(d2):
      k, l = l, k
      S_k, S_l = S_l, S_k

    middle_nodes_mapping(T_u, S_k, rule, u, k)
    middle_nodes_mapping(T_v, S_l, rule, v, l)
    return

  #case 2
  if len(middle_sk_edge) >= 2 and len(middle_tree_edge) >= 2:
    #print("middle node, middle node")
    middle_sk_node = crossing_edges(middle_sk_edge[0], middle_sk_edge[1])
    middle_tree_node = crossing_edges(middle_tree_edge[0], middle_tree_edge[1])
    middle_nodes_mapping(tree, skeleton, rule, middle_tree_node, middle_sk_node)

  #case 3
  if len(middle_sk_edge) == 1 and len(middle_tree_edge) >= 2:
    #print("middle node, middle edge")
    middle_tree_node = crossing_edges(middle_tree_edge[0], middle_tree_edge[1])
    rule[middle_tree_node] = middle_sk_edge[0]
    tree_subgraphs, sk_subgraphs = [],[]

    for subnode in tree.neighbors(middle_tree_node):
      tree_subgraphs.append(tree.subgraph(CloserSet(
          subnode, middle_tree_node, tree)))
    for i in range(2):
      sk_subgraphs.append(skeleton.subgraph(CloserSet(
          middle_sk_edge[0][i], middle_sk_edge[0][1 - i], skeleton)))
      
    tree_subgraphs.sort(key=W_CS_tree)
    sk_subgraphs.sort(key=W_CS)
    l = min(len(tree_subgraphs), len(sk_subgraphs))
    for i in range(l):
      mapping(tree_subgraphs[i], sk_subgraphs[i], rule)

    #case 4
    if len(middle_sk_edge) >= 2 and len(middle_tree_edge) == 1:
      #print("middle edge, middle node")
      middle_sk_node = crossing_edges(middle_sk_edge[0], middle_sk_edge[1])
      rule[middle_tree_edge[0]] = middle_sk_node
      tree_subgraphs, sk_subgraphs = [],[]

      for i in range(2):
        tree_subgraphs.append(tree.subgraph(CloserSet(
          middle_tree_edge[0][i], middle_tree_edge[0][1 - i], tree)))
      
      for subnode in skeleton.neighbors(middle_sk_node):
        sk_subgraphs.append(skeleton.subgraph(CloserSet(
          subnode, middle_sk_node, skeleton)))
      
      tree_subgraphs.sort(key=W_CS_tree)
      sk_subgraphs.sort(key=W_CS)
      l = min(len(tree_subgraphs), len(sk_subgraphs))
      for i in range(l):
        mapping(tree_subgraphs[i], sk_subgraphs[i], rule)
      
  return

def middle_nodes_mapping(tree, skeleton, rule, middle_tree_node, middle_sk_node):
  #print(list(tree), list(skeleton))
  rule[middle_tree_node] = middle_sk_node
  tree_subgraphs, sk_subgraphs = [], []

  for subnode in tree.neighbors(middle_tree_node):
    tree_subgraphs.append(tree.subgraph(CloserSet(
        subnode, middle_tree_node, tree)))
  for subnode in skeleton.neighbors(middle_sk_node):
    sk_subgraphs.append(skeleton.subgraph(CloserSet(
        subnode, middle_sk_node, skeleton)))
  
  l = min(len(tree_subgraphs), len(sk_subgraphs))
  if l == 0:
    return

  tree_subgraphs.sort(key=W_CS_tree, reverse=True)
  sk_subgraphs.sort(key=W_CS, reverse=True)

  for i in range(l):
    mapping(tree_subgraphs[i], sk_subgraphs[i], rule)

  return

def F(x):
  S = 0.0
  lst = (0,1)
  n = len(x) // 2
  for key in R.keys():  #проверка рекомендаций
    v1, v2 = [0,0], [0,0]
    if type(key) == type(lst):
      v1[0] = x[2 * key[0]] + x[2 * key[1]]
      v1[1] = x[2 * key[0] + 1] + x[2 * key[1] + 1]
    else:
      v1[0] = x[2 * key]
      v1[1] = x[2 * key + 1]
    value = R[key]
    if type(value) == type(lst):
      v1[0] = (Skeleton.nodes[value[0]]['pos'][0] + Skeleton.nodes[value[1]]['pos'][0]) / 2
      v1[1] = (Skeleton.nodes[value[0]]['pos'][1] + Skeleton.nodes[value[1]]['pos'][1]) / 2
    else:
      v2 = Skeleton.nodes[value]['pos'].copy()

    dv = len_e([v1[0] - v2[0], v1[1] - v2[1]])
    S += 10 * dv * dv
  
  for i in range(n):  #проверка нахождения внутри полигона
    p = [x[2 * i], x[2 * i + 1]]
    S += into_border(p)
  
  for v in range(n):  #проверка длины ребер для тех, кто без правила
    if v in R.keys():
      continue
    for node in Tree.neighbors(v):
      l = len_e([x[2 * v] - x[2 * nodes], x[2 * v + 1] - x[2 * node + 1]])
      S += 10 * abs(l - 1) ** 2
    print("S:", S)
  return S

def into_border(p):
  if not ((0 <= p[0] <=5 and 0 <= p[1] <= 20) or (15 <= p[0] <= 20 and 0 <= p[1] <= 20) or (0 <= p[0] <= 20 and 0 <= p[1] <= 5)):
    return 1000000
  
  return 0

def create_start_point():
  x0 = np.array([0.0] * 2 * len(Tree))
  n = 0
  for key in R.keys():
    if type(key) == type(n) and type(R[key]) == type(n):
      x0[2 * key] = Skeleton.nodes[R[key]]['pos'][0]
      x0[2 * key + 1] = Skeleton.nodes[R[key]]['pos'][1]
    if type(key) == type(n) and type(R[key]) == type((0,1)):
      value = R[key]
      x0[2 * key] = (Skeleton.nodes[value[0]]['pos'][0] + Skeleton.nodes[value[1]]['pos'][0]) / 2
      x0[2 * key + 1] = (Skeleton.nodes[value[0]]['pos'][1] + Skeleton.nodes[value[1]]['pos'][1]) / 2
  return x0

create_tree(G, Tree)

#Step a. Computing the polygon skeleton and the area of the faces
create_skeleton(Skeleton, skeleton_nodes, 
                skeleton_edges, skeleton_position)
create_Faces(Skeleton, skeleton_border)

#Step b. Computing the weights of the nodes of the skeleton
for node in Skeleton:
  Skeleton.nodes[node]['weight'] = W_IA(node)

#Step c. Computing the weights of the edges of the skeleton
for edge in Skeleton.edges():
  Skeleton.edges[edge]['weight'] = W_E(edge, Skeleton)

#Step d. Computing the weights of the edges of the tree
for edge in Tree.edges():
  Tree.edges[edge]['weight'] = W_E_tree(edge, Tree)

#Step e. Mapping the tree onto the skeleton
mapping(Tree, Skeleton, R)
print("recommendations:", R)

#Step f. Optimization
sc.optimize.show_options(solver='minimize', method='powell')
x0 = create_start_point()

result = sc.optimize.minimize(F, x0, method='powell', options={'ftol':1e-10, 'disp':True})
#print(result.x)

x = x0.copy()
if F(result.x) < F(x0):
  x = result.x.copy()

for node in Tree.nodes():
  Tree.nodes[node]['pos'][0] = x[2 * node]
  Tree.nodes[node]['pos'][1] = x[2 * node + 1]

recommendations: {0: 9, 5: (8, 7), 6: 7, 7: 8, 1: 10, 2: (11, 2), 3: 2, 4: 11}
Minimization of scalar function of one or more variables using the
modified Powell algorithm.

Options
-------
disp : bool
    Set to True to print convergence messages.
xtol : float
    Relative error in solution `xopt` acceptable for convergence.
ftol : float
    Relative error in ``fun(xopt)`` acceptable for convergence.
maxiter, maxfev : int
    Maximum allowed number of iterations and function evaluations.
    Will default to ``N*1000``, where ``N`` is the number of
    variables, if neither `maxiter` or `maxfev` is set. If both
    `maxiter` and `maxfev` are set, minimization will stop at the
    first reached.
direc : ndarray
    Initial set of direction vectors for the Powell method.
Optimization terminated successfully.
         Current function value: 10469.375000
         Iterations: 1
         Function evaluations: 555


In [182]:

draw_graph(fig, Skeleton, 'yellow')
draw_graph(fig, Tree, 'red')
draw_border(fig)

fig.update_layout(  #задание характеристик холста
  autosize=False,
  width=700,
  height=700,
  margin=dict(l=50, r=50, b=50, t=50, pad=4),
  paper_bgcolor="LightSteelBlue",
  showlegend = False
)

fig.show()